# Multiomics BMI Paper — Network Interaction Analysis with Baseline GLM and Longitudinal GEE

***by Kengo Watanabe***  

This Jupyter Notebook (with Python 3 kernel) assessed the baseline analyte–analyte interactions with MetBMI using generalized linear model (GLM), and subsequently assessed the interactions between the siginificant analyte–analyte pairs and days in program using generalized estimating equations (GEE).  

Input files:  
* Arivale baseline blood omics: 210104_Biological-BMI-paper_data-cleaning-BMI-omics_baseline-\[combiDF/metDF/protDF/chemDF\]-without-imputation_final-cohort.tsv  
* Arivale baseline/longitudinal MetBMI predictions: 220805_Multiomics-BMI-NatMed1stRevision_BMI-longitudinal-LASSO_MetBMI-FemaleMale.tsv  
* Arivale time-series BMI and blood omics: 210104_Biological-BMI-paper_data-cleaning-BMI-omics_time-series-\[bmiDF/combiDF\]-without-imputation_final-cohort.tsv  

Output figures and tables:  
* Figure 6b, 6c  
* Table for Supplementary Data 7  

Original notebook (memo for my future tracing):  
* dalek:\[JupyterLab HOME\]/220621_Multiomics-BMI-NatMedRevision/220806_Multiomics-BMI-NatMed1stRevision_LongitudinalNetworkAnalysis-ver2.ipynb  

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#For Arial font
#!conda install -c conda-forge -y mscorefonts
##-> The below was also needed in matplotlib 3.4.2
#import shutil
#import matplotlib
#shutil.rmtree(matplotlib.get_cachedir())
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
import time

from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats import multitest as multi
import matplotlib.lines as mlines

!conda list

# packages in environment at /opt/conda/envs/arivale-py3:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       1_gnu    conda-forge
analytics                 0.1                      pypi_0    pypi
argon2-cffi               21.1.0           py39h3811e60_0    conda-forge
arivale-data-interface    0.1.0                    pypi_0    pypi
async_generator           1.10                       py_0    conda-forge
atk-1.0                   2.36.0               h3371d22_4    conda-forge
attrs                     21.2.0             pyhd8ed1ab_0    conda-forge
backcall                  0.2.0              pyh9f0ad1d_0    conda-forge
backports                 1.0                        py_2    conda-forge
backports.functools_lru_cache 1.6.4              pyhd8ed1ab_0    conda-forge
biopython                 1.79             py39h3811e60_0    conda-forge
bleach 

## 1. Preprocessing for GLM

### 1-1. Prepare dataset

In [ ]:
#Import the baseline all omics without imputation
fileDir = '../210104_Biological-BMI-paper/ExportData/'
ipynbName = '210104_Biological-BMI-paper_data-cleaning-BMI-omics_'
fileName = 'baseline-combiDF-without-imputation_final-cohort.tsv'
tempDF = pd.read_csv(fileDir+ipynbName+fileName, sep='\t', dtype={'public_client_id': str})
tempDF = tempDF.set_index('public_client_id')

#Retrieve the baseline MetBMI-based class from the sex-stratified model
fileDir = './ExportData/'
ipynbName = '220805_Multiomics-BMI-NatMed1stRevision_BMI-longitudinal-LASSO_'
fileName = 'MetBMI-FemaleMale.tsv'
tempDF1 = pd.read_csv(fileDir+ipynbName+fileName, sep='\t', dtype={'public_client_id': str})
tempDF1 = tempDF1.sort_values(by=['public_client_id', 'days_in_program'], ascending=True)
tempDF1 = tempDF1.drop_duplicates('public_client_id', keep='first')
tempDF1 = tempDF1.set_index('public_client_id')
tempDF1['log_BaseMetBMI'] = np.log(tempDF1['BaseMetBMI'])
tempDF1 = tempDF1[['log_BaseMetBMI', 'BaseMetBMI_class']]
tempDF = pd.merge(tempDF, tempDF1, left_index=True, right_index=True, how='left')

#To apply the skewness reduction pipeline, revert the log-transformed metabolite values back to orignal scale
covarL = ['log_BaseBMI', 'Sex', 'BaseAge', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'Race']
fileDir = '../210104_Biological-BMI-paper/ExportData/'
ipynbName = '210104_Biological-BMI-paper_data-cleaning-BMI-omics_'
fileName = 'baseline-metDF-without-imputation_final-cohort.tsv'
tempDF1 = pd.read_csv(fileDir+ipynbName+fileName, sep='\t', dtype={'public_client_id': str})
tempDF1 = tempDF1.set_index('public_client_id')
for col_n in tempDF1.drop(columns=covarL).columns.tolist():
    tempDF[col_n] = np.e**tempDF[col_n]

display(tempDF)
#Update
combiDF_base = tempDF
covarL = ['log_BaseBMI', 'Sex', 'BaseAge', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'Race',
          'log_BaseMetBMI', 'BaseMetBMI_class']

In [ ]:
tempDF = combiDF_base#Not copy but just rename
print('Unique ID before elimination:', len(tempDF))

#Select the cohort used in the longitudinal analysis
month_threshold = 18
##Select participants who have 2 or more measuremnts in BMI
fileDir = '../210104_Biological-BMI-paper/ExportData/'
ipynbName = '210104_Biological-BMI-paper_data-cleaning-BMI-omics_'
fileName = 'time-series-bmiDF-without-imputation_final-cohort.tsv'
tempDF1 = pd.read_csv(fileDir+ipynbName+fileName, sep='\t', dtype={'public_client_id': str})
tempDF1 = tempDF1.set_index('KeyIndex')
tempDF1 = tempDF1.loc[tempDF1['days_in_program'] <= 365.25/12*month_threshold]
tempS = tempDF1['public_client_id'].value_counts()
tempL1 = tempS.loc[tempS>=2].index.tolist()
##Select participants who have 2 or more measuremnts in omics
fileDir = '../210104_Biological-BMI-paper/ExportData/'
ipynbName = '210104_Biological-BMI-paper_data-cleaning-BMI-omics_'
fileName = 'time-series-combiDF-without-imputation_final-cohort.tsv'
tempDF1 = pd.read_csv(fileDir+ipynbName+fileName, sep='\t', dtype={'public_client_id': str})
tempDF1 = tempDF1.set_index('KeyIndex')
tempDF1 = tempDF1.loc[tempDF1['days_in_program'] <= 365.25/12*month_threshold]
tempS = tempDF1['public_client_id'].value_counts()
tempL2 = tempS.loc[tempS>=2].index.tolist()
##Select participants who have 2 or more measuremnts in both BMI and omics
tempL = list(set(tempL1) & set(tempL2))
print('Participants who have more than 2 measurements in both BMI and omics:', len(tempL))
tempDF = tempDF.loc[tempDF.index.isin(tempL)]
print('Unique ID after elimination:', len(tempDF))

display(tempDF)
#Update
combiDF_base = tempDF

### 1-2. Drop pseudo numeric analytes

> Some analytes in clinical labs have almost same values between participants.  
> –> Because becoming invariant values after dropping outliers, these analytes will raise errors in GLM.  
> –> Eliminate these in advance.  

In [ ]:
tempDF = combiDF_base#Not copy but just rename
print('nAnalytes before the step: ', len(tempDF.drop(columns=covarL).columns))
t_start = time.time()
for col_n in tempDF.drop(columns=covarL).columns.tolist():
    #Drop category level <15 (cf.) The category level of 'race' is 14.
    if len(tempDF[col_n].unique())<15:
        tempDF = tempDF.drop(columns=[col_n])
t_elapsed = time.time() - t_start
print('Elapsed time:', round(t_elapsed//60), 'min', round(t_elapsed%60, 1), 'sec')
print('nAnalytes after the step: ', len(tempDF.drop(columns=covarL).columns))
combiDF_base = tempDF#Update the original

### 1-3. Replace zero/negative value with NaN

> Some analytes were measured as meaningful zeros in clinical labs. Also, analytes in proteomics could be negative values after the batch correction.  
> –> Skip this step.  

In [ ]:
#Check zero or negative values
tempDF = combiDF_base.drop(columns=covarL)
print('Analytes having zero or negative values')
tempS = tempDF.min(axis=0)
display(tempDF.describe(include='all').loc[:, tempS.loc[tempS<=0].index])

### 1-4. Drop outliers (>3SD from MEAN)

In [ ]:
#Check
print('Before the step:')
tempDF = combiDF_base.drop(columns=covarL)
display(tempDF.describe())
#Distribution of examples
sns.set(style='ticks', font='Arial', context='paper')
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(15, 6), sharex=False, sharey=False)
for col_i, ax in enumerate(axes.flat):
    sns.distplot(tempDF.iloc[:, col_i].dropna(), color='g', ax=ax)
    ax.set_title(tempDF.columns[col_i], fontsize='large')
sns.despine()
plt.setp(axes, xlabel='', ylabel='')
plt.setp(axes[1, :], xlabel='Raw value')
plt.setp(axes[:, 0], ylabel='Density')
fig.tight_layout()
plt.show()

In [ ]:
tempDF = combiDF_base#Not copy but just rename
t_start = time.time()
#Replace more than 3SD from MEAN with NaN
for col_n in tempDF.drop(columns=covarL).columns.tolist():
    col_mean = tempDF[col_n].mean()
    col_sd = tempDF[col_n].std(ddof=1)#Sample standard deviation: devided with N-1
    tempL = []
    for row_n in tempDF.index.tolist():
        value = tempDF.at[row_n, col_n]
        if (value > (col_mean - 3*col_sd)) & (value < (col_mean + 3*col_sd)):
            tempL.append(value)
        else:#outlier
            tempL.append(np.nan)
    tempDF[col_n] = tempL
t_elapsed = time.time() - t_start
print('Elapsed time:', round(t_elapsed//60), 'min', round(t_elapsed%60, 1), 'sec')
combiDF_base = tempDF#Update the original

In [ ]:
#Check
print('After the step:')
tempDF = combiDF_base.drop(columns=covarL)
display(tempDF.describe())
#Distribution of examples
sns.set(style='ticks', font='Arial', context='paper')
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(15, 6), sharex=False, sharey=False)
for col_i, ax in enumerate(axes.flat):
    sns.distplot(tempDF.iloc[:, col_i].dropna(), color='g', ax=ax)
    ax.set_title(tempDF.columns[col_i], fontsize='large')
sns.despine()
plt.setp(axes, xlabel='', ylabel='')
plt.setp(axes[1, :], xlabel='Raw value')
plt.setp(axes[:, 0], ylabel='Density')
fig.tight_layout()
plt.show()

### 1-5. Drop analytes with small sample size

In [ ]:
tempDF = combiDF_base#Not copy but just rename
print('nAnalytes before the step: ', len(tempDF.drop(columns=covarL).columns))
t_start = time.time()
for col_n in tempDF.drop(columns=covarL).columns.tolist():
    tempS = tempDF[col_n].dropna()
    if len(tempS) < 100:
        #Drop this analyte as one with small sample size
        tempDF = tempDF.drop(columns=[col_n])
t_elapsed = time.time() - t_start
print('Elapsed time:', round(t_elapsed//60), 'min', round(t_elapsed%60, 1), 'sec')
print('nAnalytes after the step: ', len(tempDF.drop(columns=covarL).columns))
combiDF_base = tempDF#Update the original

### 1-6. Transform to less skewed distribution

In [ ]:
#Check
print('Before the step:')
tempDF = combiDF_base.drop(columns=covarL)
print('Confirm total number of NaNs in DF:', tempDF.isnull().to_numpy().sum(axis=None))
#Skewness
tempL = []
for col_i in range(len(tempDF.columns)):
    tempL.append(stats.skew(tempDF.iloc[:, col_i].dropna()))
tempS = pd.Series(tempL, index=tempDF.columns, name='Skewness')
display(tempS.describe())
#Distribution and probability plot of examples
sns.set(style='ticks', font='Arial', context='paper')
fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(15, 12), sharex=False, sharey=False)
for ax_i, ax in enumerate(axes.flat):
    if (ax_i//5)%2 == 0:
        col_i = (ax_i//10)*5 + ax_i%5
        sns.set(style='ticks', font='Arial', context='paper')
        sns.distplot(tempDF.iloc[:, col_i].dropna(), color='g', ax=ax)
        sns.despine()
        ax.set_title(tempDF.columns[col_i], fontsize='large')
        ax.set_xlabel('Raw value')
    else:
        sns.set(style='whitegrid', font='Arial', context='paper')
        col_i = (ax_i//10)*5 + ax_i%5
        stats.probplot(tempDF.iloc[:, col_i].dropna(), plot=ax)
        ax.get_lines()[0].set_markerfacecolor('g')
        ax.get_lines()[0].set_markeredgecolor('g')
        ax.get_lines()[1].set_color('k')
        ax.get_lines()[1].set_linewidth(3)
        skewness = tempS.loc[tempDF.columns[col_i]]
        ax.set_title('Skewness: '+str(round(skewness, 3)), fontsize='large')
fig.tight_layout()
plt.show()

In [ ]:
tempDF = combiDF_base#Not copy but just rename
tempL = tempDF.drop(columns=covarL).columns.tolist()

#Prepare analyte category list to check transformation per each omics
fileDir = '../210104_Biological-BMI-paper/ExportData/'
ipynbName = '210104_Biological-BMI-paper_data-cleaning-BMI-omics_'
fileName = 'baseline-metDF-without-imputation_final-cohort.tsv'
tempDF1 = pd.read_csv(fileDir+ipynbName+fileName, sep='\t', dtype={'public_client_id': str})
tempDF1 = tempDF1.set_index('public_client_id')
tempL1 = tempDF1.loc[:, ~tempDF1.columns.isin(covarL)].columns.tolist()
tempL1 = list(set(tempL) & set(tempL1))
fileName = 'baseline-protDF-without-imputation_final-cohort.tsv'
tempDF1 = pd.read_csv(fileDir+ipynbName+fileName, sep='\t', dtype={'public_client_id': str})
tempDF1 = tempDF1.set_index('public_client_id')
tempL2 = tempDF1.loc[:, ~tempDF1.columns.isin(covarL)].columns.tolist()
tempL2 = list(set(tempL) & set(tempL2))
fileName = 'baseline-chemDF-without-imputation_final-cohort.tsv'
tempDF1 = pd.read_csv(fileDir+ipynbName+fileName, sep='\t', dtype={'public_client_id': str})
tempDF1 = tempDF1.set_index('public_client_id')
tempL3 = tempDF1.loc[:, ~tempDF1.columns.isin(covarL)].columns.tolist()
tempL3 = list(set(tempL) & set(tempL3))
tempD = {'Metabolomics':tempL1, 'Proteomics':tempL2, 'Clinical labs':tempL3}

for omics in tempD.keys():
    t_start = time.time()
    print(omics)
    #Transform to less skewed distribution
    counter_r1 = 0
    counter_r2 = 0
    counter_r3 = 0
    counter_l1 = 0
    counter_l2 = 0
    counter_l3 = 0
    counter_l4 = 0
    counter_l5 = 0
    counter_l6 = 0
    for col_n in tempD[omics]:
        tempS = tempDF[col_n].copy(deep=True)
        col_skew_0 = stats.skew(tempS.dropna())
        ##Shift the distribution to avoid NaN transformation for zero or negative values
        tempS = tempS - tempS.min() + 1
        if col_skew_0 > 0:#Right/positive skewed distribution
            #Transformation
            tempS1 = np.log(tempS)#Assumed for extremely skewed
            tempS2 = np.cbrt(tempS)#Assumed for strongly skewed
            tempS3 = np.sqrt(tempS)#Assumed for skewed
            #Evaluation
            col_skew_1 = stats.skew(tempS1.dropna())
            col_skew_2 = stats.skew(tempS2.dropna())
            col_skew_3 = stats.skew(tempS3.dropna())
            col_skew_min = np.min([abs(col_skew_0), abs(col_skew_1), abs(col_skew_2), abs(col_skew_3)])
            if col_skew_min == abs(col_skew_1):
                tempDF[col_n] = tempS1
                counter_r1 += 1
            elif col_skew_min == abs(col_skew_2):
                tempDF[col_n] = tempS2
                counter_r2 += 1
            elif col_skew_min == abs(col_skew_3):
                tempDF[col_n] = tempS3
                counter_r3 += 1
        elif col_skew_0 < 0:#Left/negative skewed distribution
            #Version1
            ##Transformation
            tempS1 = np.e**tempS#Assumed for extremely skewed
            tempS2 = tempS**3#Assumed for strongly skewed
            tempS3 = tempS**2#Assumed for skewed
            #Version2
            ##Flip horizontally to transform to the right skewed
            col_max = tempS.max()
            col_min = tempS.min()
            tempS = (col_max + col_min) - tempS
            ##Transformation
            tempS4 = np.log(tempS)#Assumed for extremely skewed
            tempS5 = np.cbrt(tempS)#Assumed for strongly skewed
            tempS6 = np.sqrt(tempS)#Assumed for skewed
            #Evaluation for both version 1 and 2
            col_skew_1 = stats.skew(tempS1.dropna())
            col_skew_2 = stats.skew(tempS2.dropna())
            col_skew_3 = stats.skew(tempS3.dropna())
            col_skew_4 = stats.skew(tempS4.dropna())
            col_skew_5 = stats.skew(tempS5.dropna())
            col_skew_6 = stats.skew(tempS6.dropna())
            col_skew_min = np.min([abs(col_skew_0), abs(col_skew_1), abs(col_skew_2), abs(col_skew_3),
                                   abs(col_skew_4), abs(col_skew_5), abs(col_skew_6)])
            if col_skew_min == abs(col_skew_1):
                tempDF[col_n] = tempS1
                counter_l1 += 1
            elif col_skew_min == abs(col_skew_2):
                tempDF[col_n] = tempS2
                counter_l2 += 1
            elif col_skew_min == abs(col_skew_3):
                tempDF[col_n] = tempS3
                counter_l3 += 1
            elif col_skew_min == abs(col_skew_4):
                #Flip horizontally again to maintain the direction of relationship
                col_max = tempS4.max()
                col_min = tempS4.min()
                tempDF[col_n] = (col_max + col_min) - tempS4
                counter_l4 += 1
            elif col_skew_min == abs(col_skew_5):
                #Flip horizontally again to maintain the direction of relationship
                col_max = tempS5.max()
                col_min = tempS5.min()
                tempDF[col_n] = (col_max + col_min) - tempS5
                counter_l5 += 1
            elif col_skew_min == abs(col_skew_6):
                #Flip horizontally again to maintain the direction of relationship
                col_max = tempS6.max()
                col_min = tempS6.min()
                tempDF[col_n] = (col_max + col_min) - tempS6
                counter_l6 += 1
    print(' nAnalyte:', len(tempD[omics]))
    print(' - log transformation for right skewed:', counter_r1)
    print(' - cbrt transformation for right skewed:', counter_r2)
    print(' - sqrt transformation for right skewed:', counter_r3)
    print(' - exponential transformation for left skewed:', counter_l1)
    print(' - cube transformation for left skewed:', counter_l2)
    print(' - square transformation for left skewed:', counter_l3)
    print(' - log transformation for left skewed with mirroring:', counter_l4)
    print(' - cbrt transformation for left skewed with mirroring:', counter_l5)
    print(' - sqrt transformation for left skewed with mirroring:', counter_l6)
    print(' - no transformation:', len(tempD[omics]) - counter_r1 - counter_r2 - counter_r3 -
          counter_l1 - counter_l2 - counter_l3 - counter_l4 - counter_l5 -counter_l6)
    t_elapsed = time.time() - t_start
    print(' Elapsed time:', round(t_elapsed//60), 'min', round(t_elapsed%60, 1), 'sec')
    print('')
combiDF_base = tempDF#Update the original

In [ ]:
#Check
print('After the step:')
tempDF = combiDF_base.drop(columns=covarL)
print('Confirm total number of NaNs in DF:', tempDF.isnull().to_numpy().sum(axis=None))
#Skewness
tempL = []
for col_i in range(len(tempDF.columns)):
    tempL.append(stats.skew(tempDF.iloc[:, col_i].dropna()))
tempS = pd.Series(tempL, index=tempDF.columns, name='Skewness')
display(tempS.describe())
#Distribution and probability plot of examples
sns.set(style='ticks', font='Arial', context='paper')
fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(15, 12), sharex=False, sharey=False)
for ax_i, ax in enumerate(axes.flat):
    if (ax_i//5)%2 == 0:
        col_i = (ax_i//10)*5 + ax_i%5
        sns.set(style='ticks', font='Arial', context='paper')
        sns.distplot(tempDF.iloc[:, col_i].dropna(), color='g', ax=ax)
        sns.despine()
        ax.set_title(tempDF.columns[col_i], fontsize='large')
        ax.set_xlabel('Raw or transformed value')
    else:
        sns.set(style='whitegrid', font='Arial', context='paper')
        col_i = (ax_i//10)*5 + ax_i%5
        stats.probplot(tempDF.iloc[:, col_i].dropna(), plot=ax)
        ax.get_lines()[0].set_markerfacecolor('g')
        ax.get_lines()[0].set_markeredgecolor('g')
        ax.get_lines()[1].set_color('k')
        ax.get_lines()[1].set_linewidth(3)
        skewness = tempS.loc[tempDF.columns[col_i]]
        ax.set_title('Skewness: '+str(round(skewness, 3)), fontsize='large')
fig.tight_layout()
plt.show()

### 1-7. Prepare pairwise combinations

In [ ]:
#Prepare combination
tempDF = combiDF_base.drop(columns=covarL)
#Compute correlation matrix
tempDF1 = tempDF.corr(method='pearson')
print('nCombinations:', int(len(tempDF1)*(len(tempDF1)-1)/2))
#Extract lower triangle matrix
tempDF1 = tempDF1.where(np.tril(np.ones(tempDF1.shape), k=-1).astype(np.bool), other=np.nan)
tempDF1.index.set_names('Variable1', inplace=True)
tempDF1 = tempDF1.reset_index().melt(var_name='Variable2', value_name='Pearson_r', id_vars=['Variable1'])
tempDF1 = tempDF1.dropna()
print('nrows:', len(tempDF1))
print('|Pearson\'s r| > 0.8:', len(tempDF1.loc[abs(tempDF1['Pearson_r'])>0.8]))

#Clean
tempDF1['PairLabel'] = tempDF1['Variable1'].str.cat(tempDF1['Variable2'], sep=' vs. ')
pairDF = tempDF1.set_index('PairLabel')
display(pairDF)

### 1-8. Split DF for each model, and then drop NaN

> Sample size is different b/w analyte pairs.  
> ***–> To maximize the sample size for each model, dropping NaN is performed after splitting.***  

> ***The large number of combinations seems to halt this step due to lack of memory.***  
> –> The subsequent preprocessing steps such as standardization should be performed at each modeling.  

## 2. Baseline analyte–analyte interactions with MetBMI

### 2-1. GLM: Analyte1 ~ b0 + b1\*Analyte2 + b2\*MetBMI + b3\*Analyte2:MetBMI + b4\*Sex + b5\*Age + b6\*AncestryPCs

> Because skewness was reduced in preprocessing, use Gaussian family with an identity link function.  

In [ ]:
tempL1 = []#For sample size
tempL2 = []#For beta-coefficient
tempL3 = []#For SE of beta-coefficient
tempL4 = []#For t-statistic
tempL5 = []#For residual degrees of freedom
tempL6 = []#For P-value
t_start = time.time()
for pairs, var1, var2 in zip(pairDF.index, pairDF['Variable1'], pairDF['Variable2']):
    #Split DF for each model, and then drop NaN
    tempL = list(set(covarL) - set(['Race']))#Race can have NaN in this study and must be eliminated for dropna()
    tempL = [variable for sublist in [[var1, var2], tempL] for variable in sublist]
    tempDF = combiDF_base[tempL]
    tempDF = tempDF.dropna()
    
    #Eliminate features with small sample size
    ##-> Able to skip because analytes are already filtered with 10% missingness.
    
    #Standardization of continuous variables
    tempDF1 = tempDF.select_dtypes(include=[np.number])
    ##Z-score transformation
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
    tempA = scaler.fit_transform(tempDF1)
    tempDF1 = pd.DataFrame(data=tempA, index=tempDF1.index, columns=tempDF1.columns)
    ##Recover the categorical covariates
    tempDF2 = tempDF.select_dtypes(exclude=[np.number])
    tempDF = pd.merge(tempDF1, tempDF2, left_index=True, right_index=True, how='left')
    
    #One-hot encoding for categorical covariates
    ##-> In statsmodels, categorical variables are automatically recognized!
    
    #Generate GLM with Gaussian family
    ##Rename the variables
    tempDF = tempDF.rename(columns={var1:'Variable1', var2:'Variable2'})
    ##Fit the model using Gaussian with an identity link function
    formula = 'Variable1 ~ Variable2 * log_BaseMetBMI + C(Sex) + BaseAge + PC1 + PC2 + PC3 + PC4 + PC5'
    model = smf.glm(formula=formula, data=tempDF,
                    family=sm.families.Gaussian(sm.families.links.identity())).fit()
    
    #Save sample size
    tempL1.append(len(tempDF))
    #Save beta-coefficient of the interaction term
    tempL2.append(model.params['Variable2:log_BaseMetBMI'])
    tempL3.append(model.bse['Variable2:log_BaseMetBMI'])
    #Save t-statistic of the interaction term
    tempL4.append(model.tvalues['Variable2:log_BaseMetBMI'])
    #Save residual degrees of freedom
    tempL5.append(int(model.df_resid))
    #Save P-value of the interaction term
    tempL6.append(model.pvalues['Variable2:log_BaseMetBMI'])
t_elapsed = time.time() - t_start
print('Elapsed time for', len(pairDF), 'GLMs: ', round(t_elapsed//60), 'min', round(t_elapsed%60, 1), 'sec')

#Clean the results
pairDF['GLM_N'] = tempL1
pairDF['GLM_Bcoef'] = tempL2
pairDF['GLM_BcoefSE'] = tempL3
pairDF['GLM_tStat'] = tempL4
pairDF['GLM_DoF'] = tempL5
pairDF['GLM_Pval'] = tempL6
##P-value adjustment by using Benjamini–Hochberg method
pairDF['GLM_AdjPval'] = multi.multipletests(pairDF['GLM_Pval'], alpha=0.05, method='fdr_bh',
                                           is_sorted=False, returnsorted=False)[1]

pairDF = pairDF.sort_values(by=['GLM_Pval'], ascending=True)
display(pairDF)

#Save
fileDir = './ExportData/'
ipynbName = '220806_Multiomics-BMI-NatMed1stRevision_LongitudinalNetworkAnalysis-ver2_'
fileName = 'GLM-interaction.tsv'
pairDF.to_csv(fileDir+ipynbName+fileName, sep='\t', index=True)

### 2-2. Significant pairs

In [ ]:
#Import (just in case of time-out)
fileDir = './ExportData/'
ipynbName = '220806_Multiomics-BMI-NatMed1stRevision_LongitudinalNetworkAnalysis-ver2_'
fileName = 'GLM-interaction.tsv'
pairDF = pd.read_csv(fileDir+ipynbName+fileName, sep='\t').set_index('PairLabel')
pairDF = pairDF.loc[pairDF['GLM_AdjPval']<0.05]
tempS = set(pairDF['Variable1']) | set(pairDF['Variable2'])
print('Significant pairwise analyte interactions with MetBMI (FDR < 0.05):', len(pairDF))
print('-> nAnalytes:', len(tempS))
display(pairDF.iloc[0:30])

In [ ]:
#Example plots

for row_i in range(5):
    pairs = pairDF.index.tolist()[row_i]
    print(pairs)
    var1 = pairDF.iloc[row_i]['Variable1']
    var2 = pairDF.iloc[row_i]['Variable2']
    tempDF = combiDF_base[[var1, var2]]
    tempDF = tempDF.dropna()
    print('n =', len(tempDF))
    #Z-score transformation
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
    tempA = scaler.fit_transform(tempDF)
    tempDF = pd.DataFrame(data=tempA, index=tempDF.index, columns=tempDF.columns)
    #Add BMI class
    tempDF = pd.merge(tempDF, combiDF_base['BaseMetBMI_class'],
                      left_index=True, right_index=True, how='inner')
    #Plot
    tempD = {'Underweight':'blue', 'Normal':'green', 'Overweight':'orange', 'Obese':'red'}
    sns.set(style='ticks', font='Arial', context='notebook')
    p = sns.lmplot(data=tempDF, x=var1, y=var2,
                   hue='BaseMetBMI_class', hue_order=tempD.keys(), palette=tempD,
                   scatter_kws={'alpha':0.5, 'edgecolor':'0.3', 's':25}, height=3, aspect=1)
    p.set(xlim=(-3.5, 3.5), xticks=np.arange(-2, 2.1, 2),
          ylim=(-3.5, 3.5), yticks=np.arange(-2, 2.1, 2))
    plt.xlabel(var1+'\n('+r'$Z$'+'-score)')
    plt.ylabel(var2+'\n('+r'$Z$'+'-score)')
    plt.show()

## 3. Preprocessing for GEE

### 3-1. Prepare dataset

In [ ]:
#Import the time-series all omics without imputation
fileDir = '../210104_Biological-BMI-paper/ExportData/'
ipynbName = '210104_Biological-BMI-paper_data-cleaning-BMI-omics_'
fileName = 'time-series-combiDF-without-imputation_final-cohort.tsv'
tempDF = pd.read_csv(fileDir+ipynbName+fileName, sep='\t', dtype={'public_client_id': str})
tempDF = tempDF.set_index('KeyIndex')

#Retrieve the baseline MetBMI-based class from the sex-stratified model
fileDir = './ExportData/'
ipynbName = '220805_Multiomics-BMI-NatMed1stRevision_BMI-longitudinal-LASSO_'
fileName = 'MetBMI-FemaleMale.tsv'
tempDF1 = pd.read_csv(fileDir+ipynbName+fileName, sep='\t', dtype={'public_client_id': str})
tempDF1 = tempDF1.sort_values(by=['public_client_id', 'days_in_program'], ascending=True)
tempDF1 = tempDF1.drop_duplicates('public_client_id', keep='first')
tempDF1 = tempDF1.set_index('public_client_id')
tempDF1['log_BaseMetBMI'] = np.log(tempDF1['BaseMetBMI'])
tempDF1 = tempDF1[['log_BaseMetBMI', 'BaseMetBMI_class']]
tempDF = pd.merge(tempDF, tempDF1, left_on='public_client_id', right_index=True, how='left')

#To apply the skewness reduction pipeline, revert the log-transformed metabolite values back to orignal scale
covarL = ['log_BaseBMI', 'Sex', 'BaseAge', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'Race']
fileDir = '../210104_Biological-BMI-paper/ExportData/'
ipynbName = '210104_Biological-BMI-paper_data-cleaning-BMI-omics_'
fileName = 'baseline-metDF-without-imputation_final-cohort.tsv'
tempDF1 = pd.read_csv(fileDir+ipynbName+fileName, sep='\t', dtype={'public_client_id': str})
tempDF1 = tempDF1.set_index('public_client_id')
for col_n in tempDF1.drop(columns=covarL).columns.tolist():
    tempDF[col_n] = np.e**tempDF[col_n]

display(tempDF)
#Update
combiDF_ts = tempDF
covarL = ['public_client_id', 'days_in_program', 'Season',
          'log_BaseBMI', 'Sex', 'BaseAge', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'Race',
          'log_BaseMetBMI', 'BaseMetBMI_class']

In [ ]:
tempDF = combiDF_ts#Not copy but just rename
print('Time-series DF nrows before elimination:', len(tempDF))
print(' -> Unique ID:', len(tempDF['public_client_id'].unique()))

#Select measurements used in the longitudinal analysis
month_threshold = 18
tempDF = tempDF.loc[tempDF['days_in_program'] <= 365.25/12*month_threshold]
print('Time-series DF nrows after selecting measurements during the taget period:', len(tempDF))
print(' -> Unique ID:', len(tempDF['public_client_id'].unique()))

#Select the cohort used in the longitudinal analysis
##Select participants who have 2 or more measuremnts in BMI
fileDir = '../210104_Biological-BMI-paper/ExportData/'
ipynbName = '210104_Biological-BMI-paper_data-cleaning-BMI-omics_'
fileName = 'time-series-bmiDF-without-imputation_final-cohort.tsv'
tempDF1 = pd.read_csv(fileDir+ipynbName+fileName, sep='\t', dtype={'public_client_id': str})
tempDF1 = tempDF1.set_index('KeyIndex')
tempDF1 = tempDF1.loc[tempDF1['days_in_program'] <= 365.25/12*month_threshold]
tempS = tempDF1['public_client_id'].value_counts()
tempL1 = tempS.loc[tempS>=2].index.tolist()
##Select participants who have 2 or more measuremnts in omics
fileDir = '../210104_Biological-BMI-paper/ExportData/'
ipynbName = '210104_Biological-BMI-paper_data-cleaning-BMI-omics_'
fileName = 'time-series-combiDF-without-imputation_final-cohort.tsv'
tempDF1 = pd.read_csv(fileDir+ipynbName+fileName, sep='\t', dtype={'public_client_id': str})
tempDF1 = tempDF1.set_index('KeyIndex')
tempDF1 = tempDF1.loc[tempDF1['days_in_program'] <= 365.25/12*month_threshold]
tempS = tempDF1['public_client_id'].value_counts()
tempL2 = tempS.loc[tempS>=2].index.tolist()
##Select participants who have 2 or more measuremnts in both BMI and omics
tempL = list(set(tempL1) & set(tempL2))
print('Participants who have more than 2 measurements in both BMI and omics:', len(tempL))
tempDF = tempDF.loc[tempDF['public_client_id'].isin(tempL)]
print('Time-series DF nrows after selecting participants who have ≥2 measuremnts in both BMI and omics:',
      len(tempDF))
print(' -> Unique ID:', len(tempDF['public_client_id'].unique()))

#Use only the measurements during period presented in the final figures of LMM
tempDF = tempDF.loc[tempDF['days_in_program'] <= 365.25/12*12]
print('Time-series DF nrows after selecting measurements during the final taget period:', len(tempDF))
print(' -> Unique ID:', len(tempDF['public_client_id'].unique()))
print(' -> nParticipants in metabolically obese group:',
      len(tempDF.loc[tempDF['BaseMetBMI_class']=='Obese']['public_client_id'].unique()))

display(tempDF)
#Update
combiDF_ts = tempDF

### 3-2. Drop pseudo numeric analytes

> Some analytes in clinical labs have almost same values between participants.  
> –> Because becoming invariant values after dropping outliers, these analytes will raise errors in GEE.  
> –> Eliminate these in advance.  

In [ ]:
tempDF = combiDF_ts#Not copy but just rename
print('nAnalytes before the step: ', len(tempDF.drop(columns=covarL).columns))
t_start = time.time()
for col_n in tempDF.drop(columns=covarL).columns.tolist():
    #Drop category level <15 (cf.) The category level of 'race' is 14.
    if len(tempDF[col_n].unique())<15:
        tempDF = tempDF.drop(columns=[col_n])
t_elapsed = time.time() - t_start
print('Elapsed time:', round(t_elapsed//60), 'min', round(t_elapsed%60, 1), 'sec')
print('nAnalytes after the step: ', len(tempDF.drop(columns=covarL).columns))
combiDF_ts = tempDF#Update the original

### 3-3. Replace zero/negative value with NaN

> Some analytes were measured as meaningful zeros in clinical labs. Also, analytes in proteomics could be negative values after the batch correction.  
> –> Skip this step.  

In [ ]:
#Check zero or negative values
tempDF = combiDF_ts.drop(columns=covarL)
print('Analytes having zero or negative values')
tempS = tempDF.min(axis=0)
display(tempDF.describe(include='all').loc[:, tempS.loc[tempS<=0].index])

### 3-4. Drop outliers (>3SD from MEAN)

> Outliers are defined based on the baseline distribution.  
> –> Because only using metabolically obese group in GEE, rather consider sample size to maximize as much as possible.  
> –> By assuming that outliers occurred not participant-dependently but stochastically, use the whole distribution.  

In [ ]:
#Check
print('Before the step:')
tempDF = combiDF_ts.drop(columns=covarL)
display(tempDF.describe())
#Distribution of examples
sns.set(style='ticks', font='Arial', context='paper')
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(15, 6), sharex=False, sharey=False)
for col_i, ax in enumerate(axes.flat):
    sns.distplot(tempDF.iloc[:, col_i].dropna(), color='g', ax=ax)
    ax.set_title(tempDF.columns[col_i], fontsize='large')
sns.despine()
plt.setp(axes, xlabel='', ylabel='')
plt.setp(axes[1, :], xlabel='Raw value')
plt.setp(axes[:, 0], ylabel='Density')
fig.tight_layout()
plt.show()

In [ ]:
tempDF = combiDF_ts#Not copy but just rename
#Prepare the baseline DF
#tempDF1 = tempDF.sort_values(by=['public_client_id', 'days_in_program'], ascending=True)
#tempDF1 = tempDF1.drop_duplicates('public_client_id', keep='first')

t_start = time.time()
#Replace more than 3SD from MEAN with NaN based on the baseline distribution
#-> Based on the whole distribution
for col_n in tempDF.drop(columns=covarL).columns.tolist():
    #col_mean = tempDF1[col_n].mean()
    #col_sd = tempDF1[col_n].std(ddof=1)#Sample standard deviation: devided with N-1
    col_mean = tempDF[col_n].mean()
    col_sd = tempDF[col_n].std(ddof=1)#Sample standard deviation: devided with N-1
    tempL = []
    for row_n in tempDF.index.tolist():
        value = tempDF.at[row_n, col_n]
        if (value > (col_mean - 3*col_sd)) & (value < (col_mean + 3*col_sd)):
            tempL.append(value)
        else:#outlier
            tempL.append(np.nan)
    tempDF[col_n] = tempL
t_elapsed = time.time() - t_start
print('Elapsed time:', round(t_elapsed//60), 'min', round(t_elapsed%60, 1), 'sec')
combiDF_ts = tempDF#Update the original

In [ ]:
#Check
print('After the step:')
tempDF = combiDF_ts.drop(columns=covarL)
display(tempDF.describe())
#Distribution of examples
sns.set(style='ticks', font='Arial', context='paper')
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(15, 6), sharex=False, sharey=False)
for col_i, ax in enumerate(axes.flat):
    sns.distplot(tempDF.iloc[:, col_i].dropna(), color='g', ax=ax)
    ax.set_title(tempDF.columns[col_i], fontsize='large')
sns.despine()
plt.setp(axes, xlabel='', ylabel='')
plt.setp(axes[1, :], xlabel='Raw value')
plt.setp(axes[:, 0], ylabel='Density')
fig.tight_layout()
plt.show()

### 3-5. Drop analytes with small sample size

In [ ]:
tempDF = combiDF_ts#Not copy but just rename
print('nAnalytes before the step: ', len(tempDF.drop(columns=covarL).columns))
t_start = time.time()
for col_n in tempDF.drop(columns=covarL).columns.tolist():
    tempS = tempDF[col_n].dropna()
    if len(tempS) < 100:
        #Drop this analyte as one with small sample size
        tempDF = tempDF.drop(columns=[col_n])
t_elapsed = time.time() - t_start
print('Elapsed time:', round(t_elapsed//60), 'min', round(t_elapsed%60, 1), 'sec')
print('nAnalytes after the step: ', len(tempDF.drop(columns=covarL).columns))
combiDF_ts = tempDF#Update the original

### 3-6. Transform to less skewed distribution

In [ ]:
#Check
print('Before the step:')
tempDF = combiDF_ts.drop(columns=covarL)
print('Confirm total number of NaNs in DF:', tempDF.isnull().to_numpy().sum(axis=None))
#Skewness
tempL = []
for col_i in range(len(tempDF.columns)):
    tempL.append(stats.skew(tempDF.iloc[:, col_i].dropna()))
tempS = pd.Series(tempL, index=tempDF.columns, name='Skewness')
display(tempS.describe())
#Distribution and probability plot of examples
sns.set(style='ticks', font='Arial', context='paper')
fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(15, 12), sharex=False, sharey=False)
for ax_i, ax in enumerate(axes.flat):
    if (ax_i//5)%2 == 0:
        col_i = (ax_i//10)*5 + ax_i%5
        sns.set(style='ticks', font='Arial', context='paper')
        sns.distplot(tempDF.iloc[:, col_i].dropna(), color='g', ax=ax)
        sns.despine()
        ax.set_title(tempDF.columns[col_i], fontsize='large')
        ax.set_xlabel('Raw value')
    else:
        sns.set(style='whitegrid', font='Arial', context='paper')
        col_i = (ax_i//10)*5 + ax_i%5
        stats.probplot(tempDF.iloc[:, col_i].dropna(), plot=ax)
        ax.get_lines()[0].set_markerfacecolor('g')
        ax.get_lines()[0].set_markeredgecolor('g')
        ax.get_lines()[1].set_color('k')
        ax.get_lines()[1].set_linewidth(3)
        skewness = tempS.loc[tempDF.columns[col_i]]
        ax.set_title('Skewness: '+str(round(skewness, 3)), fontsize='large')
fig.tight_layout()
plt.show()

In [ ]:
tempDF = combiDF_ts#Not copy but just rename
tempL = tempDF.drop(columns=covarL).columns.tolist()

#Prepare analyte category list to check transformation per each omics
fileDir = '../210104_Biological-BMI-paper/ExportData/'
ipynbName = '210104_Biological-BMI-paper_data-cleaning-BMI-omics_'
fileName = 'baseline-metDF-without-imputation_final-cohort.tsv'
tempDF1 = pd.read_csv(fileDir+ipynbName+fileName, sep='\t', dtype={'public_client_id': str})
tempDF1 = tempDF1.set_index('public_client_id')
tempL1 = tempDF1.loc[:, ~tempDF1.columns.isin(covarL)].columns.tolist()
tempL1 = list(set(tempL) & set(tempL1))
fileName = 'baseline-protDF-without-imputation_final-cohort.tsv'
tempDF1 = pd.read_csv(fileDir+ipynbName+fileName, sep='\t', dtype={'public_client_id': str})
tempDF1 = tempDF1.set_index('public_client_id')
tempL2 = tempDF1.loc[:, ~tempDF1.columns.isin(covarL)].columns.tolist()
tempL2 = list(set(tempL) & set(tempL2))
fileName = 'baseline-chemDF-without-imputation_final-cohort.tsv'
tempDF1 = pd.read_csv(fileDir+ipynbName+fileName, sep='\t', dtype={'public_client_id': str})
tempDF1 = tempDF1.set_index('public_client_id')
tempL3 = tempDF1.loc[:, ~tempDF1.columns.isin(covarL)].columns.tolist()
tempL3 = list(set(tempL) & set(tempL3))
tempD = {'Metabolomics':tempL1, 'Proteomics':tempL2, 'Clinical labs':tempL3}

for omics in tempD.keys():
    t_start = time.time()
    print(omics)
    #Transform to less skewed distribution
    counter_r1 = 0
    counter_r2 = 0
    counter_r3 = 0
    counter_l1 = 0
    counter_l2 = 0
    counter_l3 = 0
    counter_l4 = 0
    counter_l5 = 0
    counter_l6 = 0
    for col_n in tempD[omics]:
        tempS = tempDF[col_n].copy(deep=True)
        col_skew_0 = stats.skew(tempS.dropna())
        ##Shift the distribution to avoid NaN transformation for zero or negative values
        tempS = tempS - tempS.min() + 1
        if col_skew_0 > 0:#Right/positive skewed distribution
            #Transformation
            tempS1 = np.log(tempS)#Assumed for extremely skewed
            tempS2 = np.cbrt(tempS)#Assumed for strongly skewed
            tempS3 = np.sqrt(tempS)#Assumed for skewed
            #Evaluation
            col_skew_1 = stats.skew(tempS1.dropna())
            col_skew_2 = stats.skew(tempS2.dropna())
            col_skew_3 = stats.skew(tempS3.dropna())
            col_skew_min = np.min([abs(col_skew_0), abs(col_skew_1), abs(col_skew_2), abs(col_skew_3)])
            if col_skew_min == abs(col_skew_1):
                tempDF[col_n] = tempS1
                counter_r1 += 1
            elif col_skew_min == abs(col_skew_2):
                tempDF[col_n] = tempS2
                counter_r2 += 1
            elif col_skew_min == abs(col_skew_3):
                tempDF[col_n] = tempS3
                counter_r3 += 1
        elif col_skew_0 < 0:#Left/negative skewed distribution
            #Version1
            ##Transformation
            tempS1 = np.e**tempS#Assumed for extremely skewed
            tempS2 = tempS**3#Assumed for strongly skewed
            tempS3 = tempS**2#Assumed for skewed
            #Version2
            ##Flip horizontally to transform to the right skewed
            col_max = tempS.max()
            col_min = tempS.min()
            tempS = (col_max + col_min) - tempS
            ##Transformation
            tempS4 = np.log(tempS)#Assumed for extremely skewed
            tempS5 = np.cbrt(tempS)#Assumed for strongly skewed
            tempS6 = np.sqrt(tempS)#Assumed for skewed
            #Evaluation for both version 1 and 2
            col_skew_1 = stats.skew(tempS1.dropna())
            col_skew_2 = stats.skew(tempS2.dropna())
            col_skew_3 = stats.skew(tempS3.dropna())
            col_skew_4 = stats.skew(tempS4.dropna())
            col_skew_5 = stats.skew(tempS5.dropna())
            col_skew_6 = stats.skew(tempS6.dropna())
            col_skew_min = np.min([abs(col_skew_0), abs(col_skew_1), abs(col_skew_2), abs(col_skew_3),
                                   abs(col_skew_4), abs(col_skew_5), abs(col_skew_6)])
            if col_skew_min == abs(col_skew_1):
                tempDF[col_n] = tempS1
                counter_l1 += 1
            elif col_skew_min == abs(col_skew_2):
                tempDF[col_n] = tempS2
                counter_l2 += 1
            elif col_skew_min == abs(col_skew_3):
                tempDF[col_n] = tempS3
                counter_l3 += 1
            elif col_skew_min == abs(col_skew_4):
                #Flip horizontally again to maintain the direction of relationship
                col_max = tempS4.max()
                col_min = tempS4.min()
                tempDF[col_n] = (col_max + col_min) - tempS4
                counter_l4 += 1
            elif col_skew_min == abs(col_skew_5):
                #Flip horizontally again to maintain the direction of relationship
                col_max = tempS5.max()
                col_min = tempS5.min()
                tempDF[col_n] = (col_max + col_min) - tempS5
                counter_l5 += 1
            elif col_skew_min == abs(col_skew_6):
                #Flip horizontally again to maintain the direction of relationship
                col_max = tempS6.max()
                col_min = tempS6.min()
                tempDF[col_n] = (col_max + col_min) - tempS6
                counter_l6 += 1
    print(' nAnalyte:', len(tempD[omics]))
    print(' - log transformation for right skewed:', counter_r1)
    print(' - cbrt transformation for right skewed:', counter_r2)
    print(' - sqrt transformation for right skewed:', counter_r3)
    print(' - exponential transformation for left skewed:', counter_l1)
    print(' - cube transformation for left skewed:', counter_l2)
    print(' - square transformation for left skewed:', counter_l3)
    print(' - log transformation for left skewed with mirroring:', counter_l4)
    print(' - cbrt transformation for left skewed with mirroring:', counter_l5)
    print(' - sqrt transformation for left skewed with mirroring:', counter_l6)
    print(' - no transformation:', len(tempD[omics]) - counter_r1 - counter_r2 - counter_r3 -
          counter_l1 - counter_l2 - counter_l3 - counter_l4 - counter_l5 -counter_l6)
    t_elapsed = time.time() - t_start
    print(' Elapsed time:', round(t_elapsed//60), 'min', round(t_elapsed%60, 1), 'sec')
    print('')
combiDF_ts = tempDF#Update the original

In [ ]:
#Check
print('After the step:')
tempDF = combiDF_ts.drop(columns=covarL)
print('Confirm total number of NaNs in DF:', tempDF.isnull().to_numpy().sum(axis=None))
#Skewness
tempL = []
for col_i in range(len(tempDF.columns)):
    tempL.append(stats.skew(tempDF.iloc[:, col_i].dropna()))
tempS = pd.Series(tempL, index=tempDF.columns, name='Skewness')
display(tempS.describe())
#Distribution and probability plot of examples
sns.set(style='ticks', font='Arial', context='paper')
fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(15, 12), sharex=False, sharey=False)
for ax_i, ax in enumerate(axes.flat):
    if (ax_i//5)%2 == 0:
        col_i = (ax_i//10)*5 + ax_i%5
        sns.set(style='ticks', font='Arial', context='paper')
        sns.distplot(tempDF.iloc[:, col_i].dropna(), color='g', ax=ax)
        sns.despine()
        ax.set_title(tempDF.columns[col_i], fontsize='large')
        ax.set_xlabel('Raw or transformed value')
    else:
        sns.set(style='whitegrid', font='Arial', context='paper')
        col_i = (ax_i//10)*5 + ax_i%5
        stats.probplot(tempDF.iloc[:, col_i].dropna(), plot=ax)
        ax.get_lines()[0].set_markerfacecolor('g')
        ax.get_lines()[0].set_markeredgecolor('g')
        ax.get_lines()[1].set_color('k')
        ax.get_lines()[1].set_linewidth(3)
        skewness = tempS.loc[tempDF.columns[col_i]]
        ax.set_title('Skewness: '+str(round(skewness, 3)), fontsize='large')
fig.tight_layout()
plt.show()

### 3-7. Prepare the significant pairwise combinations

In [ ]:
#Import (just in case of time-out)
fileDir = './ExportData/'
ipynbName = '220806_Multiomics-BMI-NatMed1stRevision_LongitudinalNetworkAnalysis-ver2_'
fileName = 'GLM-interaction.tsv'
pairDF = pd.read_csv(fileDir+ipynbName+fileName, sep='\t').set_index('PairLabel')
pairDF = pairDF.loc[pairDF['GLM_AdjPval']<0.05]
tempS = set(pairDF['Variable1']) | set(pairDF['Variable2'])
print('Significant pairwise analyte interactions with MetBMI (FDR < 0.05):', len(pairDF))
print('-> nAnalytes:', len(tempS))

### 3-8. Split DF for each model, and then drop NaN

> Sample size is different b/w analyte pairs.  
> ***–> To maximize the sample size for each model, dropping NaN is performed after splitting.***  

> ***The large number of combinations seems to halt this step due to lack of memory.***  
> –> The subsequent preprocessing steps such as standardization should be performed at each modeling.  

## 4. Analyte–analyte interactions with days in program in metabolically obese group

### 4-1. GEE: Analyte1 ~ b0 + b1\*Analyte2 + b2\*Days + b3\*Analyte2:Days + b4\*Sex + b5\*Age + b6\*Season + b7\*AncestryPCs

> Because of multiple measurements per participant, use GEE with exchangeable covariance structure. As well as the above GLM, use Gaussian family with an identity link function.  

In [ ]:
base_class = 'Obese'
tempL1 = []#For sample size (observations)
tempL2 = []#For the number of groups (= unique participants)
tempL3 = []#For beta-coefficient
tempL4 = []#For SE of beta-coefficient
tempL5 = []#For t-statistic
tempL6 = []#For P-value
t_start = time.time()
for pairs, var1, var2 in zip(pairDF.index, pairDF['Variable1'], pairDF['Variable2']):
    #Split DF for each model, and then drop NaN
    tempL = list(set(covarL) - set(['Race']))#Race can have NaN in this study and must be eliminated for dropna()
    tempL = [variable for sublist in [[var1, var2], tempL] for variable in sublist]
    tempDF = combiDF_ts[tempL]
    tempDF = tempDF.dropna()
    
    #Select BMI class
    tempDF = tempDF.loc[tempDF['BaseMetBMI_class']==base_class]
    
    #Standardization of continuous variables based on the whole distribution
    tempDF1 = tempDF.select_dtypes(include=[np.number])
    ##Z-score transformation
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
    tempA = scaler.fit_transform(tempDF1)
    tempDF1 = pd.DataFrame(data=tempA, index=tempDF1.index, columns=tempDF1.columns)
    ##Recover the categorical covariates
    tempDF2 = tempDF.select_dtypes(exclude=[np.number])
    tempDF = pd.merge(tempDF1, tempDF2, left_index=True, right_index=True, how='left')
    
    #One-hot encoding for categorical covariates
    ##-> In statsmodels, categorical variables are automatically recognized!
    
    #Generate GEE with exchangeable covariance structure
    ##Rename the variables
    tempDF = tempDF.rename(columns={var1:'Variable1', var2:'Variable2'})
    ##Fit the model using exchangeable covariance structure
    formula = 'Variable1 ~ Variable2 * days_in_program + C(Sex) + BaseAge + C(Season) + PC1 + PC2 + PC3 + PC4 + PC5'
    model = sm.GEE.from_formula(formula=formula, data=tempDF, groups='public_client_id',
                                family=sm.families.Gaussian(sm.families.links.identity()),
                                cov_struct=sm.cov_struct.Exchangeable()).fit()
    
    #Save
    #Save sample size
    tempL1.append(len(tempDF))
    #Save the number of groups
    tempL2.append(len(tempDF['public_client_id'].unique()))
    #Save beta-coefficient of the interaction term
    tempL3.append(model.params['Variable2:days_in_program'])
    tempL4.append(model.bse['Variable2:days_in_program'])
    #Save t-statistic of the interaction term
    tempL5.append(model.tvalues['Variable2:days_in_program'])
    #Save P-value of the interaction term
    tempL6.append(model.pvalues['Variable2:days_in_program'])
t_elapsed = time.time() - t_start
print('Elapsed time for', len(pairDF), 'GEEs: ', round(t_elapsed//60), 'min', round(t_elapsed%60, 1), 'sec')

#Clean the results
pairDF['GEE_nObs'] = tempL1
pairDF['GEE_nGroups'] = tempL2
pairDF['GEE_Bcoef'] = tempL3
pairDF['GEE_BcoefSE'] = tempL4
pairDF['GEE_tStat'] = tempL5
pairDF['GEE_Pval'] = tempL6
##P-value adjustment by using Benjamini–Hochberg method
pairDF['GEE_AdjPval'] = multi.multipletests(pairDF['GEE_Pval'], alpha=0.05, method='fdr_bh',
                                            is_sorted=False, returnsorted=False)[1]

pairDF = pairDF.sort_values(by=['GEE_Pval'], ascending=True)
display(pairDF)

#Save
fileDir = './ExportData/'
ipynbName = '220806_Multiomics-BMI-NatMed1stRevision_LongitudinalNetworkAnalysis-ver2_'
fileName = 'GEE-interaction.tsv'
pairDF.to_csv(fileDir+ipynbName+fileName, sep='\t', index=True)

### 4-2. Significant pairs

In [ ]:
#Import (just in case of time-out)
fileDir = './ExportData/'
ipynbName = '220806_Multiomics-BMI-NatMed1stRevision_LongitudinalNetworkAnalysis-ver2_'
fileName = 'GEE-interaction.tsv'
pairDF = pd.read_csv(fileDir+ipynbName+fileName, sep='\t').set_index('PairLabel')
pairDF = pairDF.loc[pairDF['GEE_AdjPval']<0.05]
tempS = set(pairDF['Variable1']) | set(pairDF['Variable2'])
print('Significant pairwise analyte interactions with days in program (FDR < 0.05):', len(pairDF))
print('-> nAnalytes:', len(tempS))
display(pairDF)

> If Pearson's r \* beta-coefficient in GLM > 0, the analyte correlation is positively modified by MetBMI.  
> If Pearson's r \* beta-coefficient in GEE > 0, the analyte correlation in obese group is positively modified by days.  
> –> If beta-coefficient in GLM \* beta-coefficient in GEE < 0, the association direction is concordant; that is, the analyte correlation in metabolically obese group is changed to that in normal group during the program.  

In [ ]:
#Concordant pairs
pairDF = pairDF.loc[pairDF['GLM_Bcoef']*pairDF['GEE_Bcoef'] < 0]
tempS = set(pairDF['Variable1']) | set(pairDF['Variable2'])
print('Concordant pairs:', len(pairDF))
print('-> nAnalytes:', len(tempS))
display(pairDF)

#### 4-2-1. Categorize with measurement number

In [ ]:
#Example plots

#Convert days_in_program value to categorical value
##-> Although different meaning from GEE, try measurement number.
##Ensure order
combiDF_ts = combiDF_ts.sort_values(by=['public_client_id', 'days_in_program'], ascending=True)
tempL = []
counter = 1
row_n_before = 'InitialDummyName'
for row_n in combiDF_ts['public_client_id'].tolist():
    if row_n == row_n_before:
        counter += 1
        tempL.append(counter)
        row_n_before = row_n
    else:
        counter = 1
        tempL.append(counter)
        row_n_before = row_n
combiDF_ts['MeasureNum'] = tempL

for row_i in range(np.min([15, len(pairDF)])):
    pairs = pairDF.index.tolist()[row_i]
    print(pairs)
    var1 = pairDF.iloc[row_i]['Variable1']
    var2 = pairDF.iloc[row_i]['Variable2']
    
    #GLM result
    tempDF = combiDF_base[[var1, var2]]
    tempDF = tempDF.dropna()
    print('GLM n =', len(tempDF))
    ##Z-score transformation
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
    tempA = scaler.fit_transform(tempDF)
    tempDF = pd.DataFrame(data=tempA, index=tempDF.index, columns=tempDF.columns)
    ##Add BMI class
    tempDF = pd.merge(tempDF, combiDF_base['BaseMetBMI_class'],
                      left_index=True, right_index=True, how='inner')
    ##Plot
    tempD = {'Underweight':'blue', 'Normal':'green', 'Overweight':'orange', 'Obese':'red'}
    sns.set(style='ticks', font='Arial', context='notebook')
    p = sns.lmplot(data=tempDF, x=var1, y=var2,
                   hue='BaseMetBMI_class', hue_order=tempD.keys(), palette=tempD,
                   scatter_kws={'alpha':0.5, 'edgecolor':'0.3', 's':25}, height=3, aspect=1)
    p.set(xlim=(-3.5, 3.5), xticks=np.arange(-2, 2.1, 2),
          ylim=(-3.5, 3.5), yticks=np.arange(-2, 2.1, 2))
    plt.xlabel(var1+'\n('+r'$Z$'+'-score)')
    plt.ylabel(var2+'\n('+r'$Z$'+'-score)')
    plt.show()
    
    #GEE result
    tempDF = combiDF_ts[[var1, var2, 'BaseMetBMI_class', 'MeasureNum']]
    tempDF = tempDF.dropna()
    ##Z-score transformation based on the baseline distribution
    tempDF1 = tempDF.loc[tempDF['MeasureNum']==1].drop(columns=['BaseMetBMI_class', 'MeasureNum'])#baseline
    tempDF2 = tempDF.drop(columns=['BaseMetBMI_class', 'MeasureNum'])#time-series
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
    scaler.fit(tempDF1)
    tempA = scaler.transform(tempDF2)
    tempDF2 = pd.DataFrame(data=tempA, index=tempDF2.index, columns=tempDF2.columns)
    ##Add category label
    tempDF1 = tempDF[['BaseMetBMI_class', 'MeasureNum']]
    tempDF = pd.merge(tempDF1, tempDF2, left_index=True, right_index=True, how='left')
    tempD = {1:'1st', 2:'2nd', 3:'3rd and more', 4:'3rd and more', 5:'3rd and more'}
    tempDF['Measurement'] = tempDF['MeasureNum'].map(tempD)
    ##Select obese group
    tempDF = tempDF.loc[tempDF['BaseMetBMI_class']=='Obese']
    print('GEE n =', len(tempDF))
    #Plot
    tempD = {'1st':'red', '2nd':'orange', '3rd and more':'green'}
    sns.set(style='ticks', font='Arial', context='notebook')
    p = sns.lmplot(data=tempDF, x=var1, y=var2, hue='Measurement', hue_order=tempD.keys(), palette=tempD,
                   scatter_kws={'alpha':0.5, 'edgecolor':'0.3', 's':25}, height=3, aspect=1)
    p.set(xlim=(-3.5, 3.5), xticks=np.arange(-2, 2.1, 2),
          ylim=(-3.5, 3.5), yticks=np.arange(-2, 2.1, 2))
    plt.xlabel(var1+'\n('+r'$Z$'+'-score)')
    plt.ylabel(var2+'\n('+r'$Z$'+'-score)')
    plt.show()

#### 4-2-2. Categorize with binned days

In [ ]:
#Example plots

#Convert days_in_program value to categorical value
tempL = []
for value in combiDF_ts['days_in_program'].tolist():
    if value < 365.25/12*4:
        tempL.append('0–4 month')
    elif value < 365.25/12*8:
        tempL.append('4–8 month')
    elif value <= 365.25/12*12:
        tempL.append('8–12 month')
    else:#Just in case
        tempL.append('Error?')
combiDF_ts['Period'] = tempL

for row_i in range(np.min([15, len(pairDF)])):
    pairs = pairDF.index.tolist()[row_i]
    print(pairs)
    var1 = pairDF.iloc[row_i]['Variable1']
    var2 = pairDF.iloc[row_i]['Variable2']
    
    #GLM result
    tempDF = combiDF_base[[var1, var2]]
    tempDF = tempDF.dropna()
    print('GLM n =', len(tempDF))
    ##Z-score transformation
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
    tempA = scaler.fit_transform(tempDF)
    tempDF = pd.DataFrame(data=tempA, index=tempDF.index, columns=tempDF.columns)
    ##Add BMI class
    tempDF = pd.merge(tempDF, combiDF_base['BaseMetBMI_class'],
                      left_index=True, right_index=True, how='inner')
    ##Plot
    tempD = {'Underweight':'blue', 'Normal':'green', 'Overweight':'orange', 'Obese':'red'}
    sns.set(style='ticks', font='Arial', context='notebook')
    p = sns.lmplot(data=tempDF, x=var1, y=var2,
                   hue='BaseMetBMI_class', hue_order=tempD.keys(), palette=tempD,
                   scatter_kws={'alpha':0.5, 'edgecolor':'0.3', 's':25}, height=3, aspect=1)
    p.set(xlim=(-3.5, 3.5), xticks=np.arange(-2, 2.1, 2),
          ylim=(-3.5, 3.5), yticks=np.arange(-2, 2.1, 2))
    plt.xlabel(var1+'\n('+r'$Z$'+'-score)')
    plt.ylabel(var2+'\n('+r'$Z$'+'-score)')
    plt.show()
    
    #GEE result
    tempDF = combiDF_ts[[var1, var2, 'BaseMetBMI_class', 'MeasureNum', 'Period']]
    tempDF = tempDF.dropna()
    ##Z-score transformation based on the baseline distribution
    tempDF1 = tempDF.loc[tempDF['MeasureNum']==1].drop(columns=['BaseMetBMI_class', 'MeasureNum', 'Period'])#baseline
    tempDF2 = tempDF.drop(columns=['BaseMetBMI_class', 'MeasureNum', 'Period'])#time-series
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
    scaler.fit(tempDF1)
    tempA = scaler.transform(tempDF2)
    tempDF2 = pd.DataFrame(data=tempA, index=tempDF2.index, columns=tempDF2.columns)
    ##Add category label
    tempDF1 = tempDF[['BaseMetBMI_class', 'Period']]
    tempDF = pd.merge(tempDF1, tempDF2, left_index=True, right_index=True, how='left')
    ##Select obese group
    tempDF = tempDF.loc[tempDF['BaseMetBMI_class']=='Obese']
    print('GEE n =', len(tempDF))
    #Plot
    tempD = {'0–4 month':'red', '4–8 month':'orange', '8–12 month':'green'}
    sns.set(style='ticks', font='Arial', context='notebook')
    p = sns.lmplot(data=tempDF, x=var1, y=var2, hue='Period', hue_order=tempD.keys(), palette=tempD,
                   scatter_kws={'alpha':0.5, 'edgecolor':'0.3', 's':25}, height=3, aspect=1)
    p.set(xlim=(-3.5, 3.5), xticks=np.arange(-2, 2.1, 2),
          ylim=(-3.5, 3.5), yticks=np.arange(-2, 2.1, 2))
    plt.xlabel(var1+'\n('+r'$Z$'+'-score)')
    plt.ylabel(var2+'\n('+r'$Z$'+'-score)')
    plt.show()

In [ ]:
#Save representative plots for presentation
tempL = ['2-oxoarginine* vs. alpha-hydroxyisocaproate',
         'homoarginine vs. phenyllactate (PLA)',
         '1-stearoyl-2-docosahexaenoyl-GPE (18:0/22:6)* vs. ALBUMIN']

#GLM
tempDF1 = pd.DataFrame()
for pairs in tempL:
    var1 = pairDF.at[pairs, 'Variable1']
    var2 = pairDF.at[pairs, 'Variable2']
    tempDF = combiDF_base[[var1, var2]]
    tempDF = tempDF.dropna()
    print(pairs, 'GLM: n =', len(tempDF))
    #Z-score transformation
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
    tempA = scaler.fit_transform(tempDF)
    tempDF = pd.DataFrame(data=tempA, index=tempDF.index, columns=tempDF.columns)
    #Add BMI class
    tempDF = pd.merge(tempDF, combiDF_base['BaseMetBMI_class'],
                      left_index=True, right_index=True, how='inner')
    #Merge
    tempDF['PairLabel'] = np.repeat(pairs, len(tempDF))
    tempDF['Variable1'] = np.repeat(var1, len(tempDF))
    tempDF['Variable2'] = np.repeat(var2, len(tempDF))
    tempDF = tempDF.rename(columns={var1:'Zscore1', var2:'Zscore2'})
    tempDF1 = pd.concat([tempDF1, tempDF], axis=0)
#Plot
tempD = {'Normal':'green', 'Overweight':'orange', 'Obese':'red'}
tempD1 = {tempL[0]:'Intra-metabolomics', tempL[1]:'Intra-metabolomics', tempL[2]:'Inter-omics'}
sns.set(style='ticks', font='Arial', context='talk')
fig, axes = plt.subplots(nrows=1, ncols=len(tempL),
                         figsize=(3.75*len(tempL), 3.75+0.75), sharex=False, sharey=False)
axis_xymin = -3.5
axis_xymax = 3.5
xymin = -2.0
xymax = 2.0
xyinter = 2.0
##Set axis range first; otherwise, regression line can be truncated differently
plt.setp(axes, xlim=(axis_xymin, axis_xymax), xticks=np.arange(xymin, xymax+xyinter/10, xyinter))
plt.setp(axes, ylim=(axis_xymin, axis_xymax), yticks=np.arange(xymin, xymax+xyinter/10, xyinter))
for ax_i, ax in enumerate(axes.flat):
    pairs = tempL[ax_i]
    #Prepare DF
    tempDF = tempDF1.loc[tempDF1['PairLabel']==pairs]
    #Scatterplot with regression line
    for bbmi_class in tempD.keys():
        tempDF2 = tempDF.loc[tempDF['BaseMetBMI_class']==bbmi_class]
        sns.regplot(data=tempDF2, x='Zscore1', y='Zscore2', color=tempD[bbmi_class],
                    scatter=True, fit_reg=True, ci=95, truncate=False, marker='o',
                    scatter_kws={'alpha':0.5, 'edgecolor':'0.3', 's':25}, ax=ax)
    #Axis label
    tempDF = tempDF.reset_index()[['PairLabel', 'Variable1', 'Variable2']]
    tempDF = tempDF.drop_duplicates('PairLabel', keep='first').set_index('PairLabel')
    var1 = tempDF.at[pairs, 'Variable1']
    var2 = tempDF.at[pairs, 'Variable2']
    ax.set_xlabel(var1.replace('-GPE', '-\nGPE')+'\n('+r'$Z$'+'-score)')
    ax.set_ylabel(var2.replace('-GPE', '-\nGPE')+'\n('+r'$Z$'+'-score)')
    #Facet title
    ax.set_title(tempD1[pairs], {'fontsize':'large'})
sns.despine()
plt.tight_layout()
#Generate legend; default point in lmplot is tiny
tempL1 = []
for bmi_class in tempD.keys():
    tempL1.append(mlines.Line2D([], [], color=tempD[bmi_class], label=bmi_class, linewidth=4))
plt.legend(handles=tempL1, fontsize='medium', title='Baseline MetBMI class', title_fontsize='large',
           bbox_to_anchor=(1, 0.5), loc='center left', borderaxespad=3, frameon=True)
##Save
fileDir = './ExportFigures/'
ipynbName = '220806_Multiomics-BMI-NatMed1stRevision_LongitudinalNetworkAnalysis-ver2_'
fileName = 'GLM-interaction-examples.tif'
plt.gcf().savefig(fileDir+ipynbName+fileName, dpi=300, bbox_inches='tight', pad_inches=0.04,
                  pil_kwargs={'compression':'tiff_lzw'})
plt.show()

#GEE result
tempDF1 = pd.DataFrame()
for pairs in tempL:
    var1 = pairDF.at[pairs, 'Variable1']
    var2 = pairDF.at[pairs, 'Variable2']
    tempDF = combiDF_ts[[var1, var2, 'BaseMetBMI_class', 'MeasureNum', 'Period']]
    tempDF = tempDF.dropna()
    #Z-score transformation based on the baseline distribution
    tempDF2 = tempDF.loc[tempDF['MeasureNum']==1].drop(columns=['BaseMetBMI_class', 'MeasureNum', 'Period'])#baseline
    tempDF3 = tempDF.drop(columns=['BaseMetBMI_class', 'MeasureNum', 'Period'])#time-series
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
    scaler.fit(tempDF2)
    tempA = scaler.transform(tempDF3)
    tempDF3 = pd.DataFrame(data=tempA, index=tempDF3.index, columns=tempDF3.columns)
    #Add category label
    tempDF2 = tempDF[['BaseMetBMI_class', 'Period']]
    tempDF = pd.merge(tempDF2, tempDF3, left_index=True, right_index=True, how='left')
    #Select obese group
    tempDF = tempDF.loc[tempDF['BaseMetBMI_class']=='Obese']
    print(pairs, 'GEE: n =', len(tempDF))
    #Merge
    tempDF['PairLabel'] = np.repeat(pairs, len(tempDF))
    tempDF['Variable1'] = np.repeat(var1, len(tempDF))
    tempDF['Variable2'] = np.repeat(var2, len(tempDF))
    tempDF = tempDF.rename(columns={var1:'Zscore1', var2:'Zscore2'})
    tempDF1 = pd.concat([tempDF1, tempDF], axis=0)
#Plot
tempD = {'0–4 month':'red', '4–8 month':'orange', '8–12 month':'green'}
tempD1 = {tempL[0]:'Intra-metabolomics', tempL[1]:'Intra-metabolomics', tempL[2]:'Inter-omics'}
sns.set(style='ticks', font='Arial', context='talk')
fig, axes = plt.subplots(nrows=1, ncols=len(tempL),
                         figsize=(3.75*len(tempL), 3.75+0.75), sharex=False, sharey=False)
axis_xymin = -3.5
axis_xymax = 3.5
xymin = -2.0
xymax = 2.0
xyinter = 2.0
##Set axis range first; otherwise, regression line can be truncated differently
plt.setp(axes, xlim=(axis_xymin, axis_xymax), xticks=np.arange(xymin, xymax+xyinter/10, xyinter))
plt.setp(axes, ylim=(axis_xymin, axis_xymax), yticks=np.arange(xymin, xymax+xyinter/10, xyinter))
for ax_i, ax in enumerate(axes.flat):
    pairs = tempL[ax_i]
    #Prepare DF
    tempDF = tempDF1.loc[tempDF1['PairLabel']==pairs]
    #Scatterplot with regression line
    for period in tempD.keys():
        tempDF2 = tempDF.loc[tempDF['Period']==period]
        sns.regplot(data=tempDF2, x='Zscore1', y='Zscore2', color=tempD[period],
                    scatter=True, fit_reg=True, ci=95, truncate=False, marker='o',
                    scatter_kws={'alpha':0.5, 'edgecolor':'0.3', 's':25}, ax=ax)
    #Axis label
    tempDF = tempDF.reset_index()[['PairLabel', 'Variable1', 'Variable2']]
    tempDF = tempDF.drop_duplicates('PairLabel', keep='first').set_index('PairLabel')
    var1 = tempDF.at[pairs, 'Variable1']
    var2 = tempDF.at[pairs, 'Variable2']
    ax.set_xlabel(var1.replace('-GPE', '-\nGPE')+'\n('+r'$Z$'+'-score)')
    ax.set_ylabel(var2.replace('-GPE', '-\nGPE')+'\n('+r'$Z$'+'-score)')
    #Facet title
    ax.set_title(tempD1[pairs], {'fontsize':'large'})
sns.despine()
plt.tight_layout()
#Generate legend; default point in lmplot is tiny
tempL1 = []
for bmi_class in tempD.keys():
    tempL1.append(mlines.Line2D([], [], color=tempD[bmi_class], label=bmi_class, linewidth=4))
plt.legend(handles=tempL1, fontsize='medium', title='Period in program', title_fontsize='large',
           bbox_to_anchor=(1, 0.5), loc='center left', borderaxespad=3, frameon=True)
##Save
fileDir = './ExportFigures/'
ipynbName = '220806_Multiomics-BMI-NatMed1stRevision_LongitudinalNetworkAnalysis-ver2_'
fileName = 'GEE-interaction-examples.tif'
plt.gcf().savefig(fileDir+ipynbName+fileName, dpi=300, bbox_inches='tight', pad_inches=0.04,
                  pil_kwargs={'compression':'tiff_lzw'})
plt.show()

# — End of notebook —